# BM25代码实践
Milvus版本>=2.4.0
marshmallow<4.0

如果不对，重新安装，并且一定要记得重启内核，重启内核，不然还会用旧的！

In [1]:
!pip install "marshmallow<4.0" --force-reinstall --quiet

DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.0 requires typer<0.10.0,>=0.3.0, but you have typer 0.16.0 which is incompatible.
pai-easycv 0.11.7 requires timm==0.5.4, but you have timm 1.0.19 which is incompatible.
ms-swift 3.7.1 requires peft<0.17,>=0.11, but you have peft 0.17.0 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.16.0 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update,

In [2]:
!pip install pymilvus==2.4.7 torch --quiet

DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.0 requires typer<0.10.0,>=0.3.0, but you have typer 0.16.0 which is incompatible.
ms-swift 3.7.1 requires peft<0.17,>=0.11, but you have peft 0.17.0 which is incompatible.

[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install "pymilvus[model]" --quiet

DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


假设下面每一个字符串都是一个文档

In [1]:
docs = [
    "机器学习正在改变我们的生活方式。",
    "深度学习在图像识别中表现出色。",
    "自然语言处理是计算机科学的重要领域。",
    "自动驾驶依赖于先进的算法。",
    "AI可以帮助医生诊断疾病。",
    "金融领域广泛应用数据分析技术。",
    "生产效率可以通过自动化技术提高。",
    "机器智能的未来充满潜力。",
    "大数据支持是机器智能发展的关键。",
    "量子隧穿效应使得电子能够穿过经典力学认为无法穿过的势垒，这在半导体器件中有着重要的应用。"
]

使用BM25对第一个文档“机器学习正在改变我们的生活方式”做分词处理

In [ ]:
from pymilvus.model.sparse.bm25.tokenizers import build_default_analyzer
from pymilvus.model.sparse import BM25EmbeddingFunction

# 使用支持中文的分析器
analyzer = build_default_analyzer(language="zh")

# 分析器对文本做分词处理
tokens1 = analyzer(docs[0])
print(tokens1)

分词结果如下：

['机器', '学习', '改变', '生活', '方式']

接下来对整个文档做分词处理，并且计算文档集合的IDF等参数：

In [8]:
# 创建BM25EmbeddingFunction实例，传入分词器，以及其他参数
bm25_ef = BM25EmbeddingFunction(analyzer)

# 计算文档集合的参数
bm25_ef.fit(docs)

# 保存训练好的参数到磁盘以加快后续处理
bm25_ef.save("bm25_params.json")

查看参数具体内容

In [ ]:
import json

file_path = "bm25_params.json"
with open(file_path, 'r', encoding='utf-8') as file:
    bm25_params = json.load(file)
    print(bm25_params)

corpus_size是文档数量，可以看到avgdl和idf_value两个参数

idf_word 是 BM25对文档集合的分词结果，也就是前面提到的词汇表。词汇表中单词的数量，也是稀疏向量的维度。

In [ ]:
# BM25词汇表中的单词数量
print(f"BM25词汇表中的单词数量：{len(bm25_params['idf_word'])}")

# BM25稀疏向量的维度
print(f"BM25稀疏向量维度：{bm25_ef.dim}")

返回的结果：

BM25词汇表中的单词数量：46
BM25稀疏向量维度：46

需要的参数计算好了，接下来就可以生成文档集合的稀疏向量了。文档集合中有10篇文档，也就是10个字符串，而稀疏向量的维度是46，所以文档集合的稀疏向量是一个10行46列的矩阵。每一行表示一个文档的稀疏向量。

In [ ]:
# 生成文档集合的稀疏向量
sparse_vectors_bm25 = bm25_ef.encode_documents(docs)

# 打印文档集合的稀疏向量
print(sparse_vectors_bm25)

我们来看下第一个文档“机器学习正在改变我们的生活方式。”的稀疏向量：

In [ ]:
# 第一个文档的稀疏向量
print(list(sparse_vectors_bm25)[0])

结果为：

(0, 0)        1.0344827586206897
  (0, 1)        1.0344827586206897
  (0, 2)        1.0344827586206897
  (0, 3)        1.0344827586206897
  (0, 4)        1.0344827586206897
你发现了吧，第一个文档的稀疏向量只有5个非零元素，因为它的分词结果是5个单词，对应上了。而且，每个元素的值都相同，说明它们的逆文档频率 IDF 和词频 TF 都是一样的。

第一个文档的分词结果：

['机器', '学习', '改变', '生活', '方式']
文档集合处理好了，我们再给出一个查询的句子，就可以执行搜索了。

In [ ]:
query = ["自动驾驶如何影响汽车行业？"]

# 把查询文本向量化
query_sparse_vectors_bm25 = bm25_ef.encode_queries(query)

# 打印稀疏向量
print(query_sparse_vectors_bm25)

# 查询的分词结果
print(analyzer(query[0]))

查看查询的稀疏向量，以及它的分词结果。

(0, 12)        1.845826690498331
  (0, 13)        1.845826690498331
['自动', '驾驶', '影响', '汽车行业']
你可能会有疑问，为什么查询分词后得到4个单词，但是它的稀疏向量只有2维？因为这4个单词中，词汇表中只有“自动”和“驾驶”，没有“影响”和“汽车行业”，后两个词的 BM25 分数为0。

哎，毕竟是门外汉啊。